In [1]:
import numpy as np
import sensors
import ecm_determ

In [2]:
DIST_RATIO = 0.5

In [3]:
Num_sensors1 = 5
Num_emitters1 = 1
sample_size1 = 100
theta1_rad = [0.5] # Направление прибытия (DOA) в радианах
theta1_deg = np.rad2deg(theta1_rad) # Направление прибытия (DOA) в градусах
#Q1 = 1.1 * np.eye(Num_sensors1, dtype=np.float64) # Ковариация шума
Q1 = sensors.random_complex_cov(n=Num_sensors1, max_real=1.2, seed=50)
A1 = np.exp(-2j * np.pi * DIST_RATIO * np.arange(Num_sensors1).reshape(-1,1) * np.sin(theta1_rad)) # Матрица управляющих векторов
# Генерация векторов сигнала, шума и принятого сигнала
S1 = sensors.gds(Num_emitters1, sample_size1)
n1 = sensors.gss(Num_sensors1, sample_size1, Q1)
X1 = (A1 @ S1.T + n1.T).T

In [4]:
X1_with_mv = sensors.MCAR(X1, [2,4], [40, 40])

In [5]:
EM_theta1_rad, EM_S1, EM_Q1, lhd_1 = ecm_determ.multi_start_ECM_un(X1_with_mv, 1, num_of_starts=10, max_iter=10, rtol=1e-6)

0-th start
Initial theta = [-2.83488836]
min=-4.999627905706155
is_spd(weighted_Q) False
is_spd(Q) True
is_spd(new_Q) True
incomplete likelihood is -200.22445475318085 on iteration 0
1-th start
Initial theta = [-2.86887136]
min=-0.5891037672839443
is_spd(weighted_Q) False
is_spd(Q) True
is_spd(new_Q) True
incomplete likelihood is -444.9120621163012 on iteration 0
2-th start
Initial theta = [-0.32903623]
min=-5.008250026602847
is_spd(weighted_Q) False
is_spd(Q) True
is_spd(new_Q) True
incomplete likelihood is -130.5210863078478 on iteration 0
3-th start
Initial theta = [2.8344867]
min=-0.5331893802666425
is_spd(weighted_Q) False
is_spd(Q) True
is_spd(new_Q) True
incomplete likelihood is -166.78517150979394 on iteration 0
4-th start
Initial theta = [-2.08143765]
min=-0.7621512805322519
is_spd(weighted_Q) False
is_spd(Q) True
is_spd(new_Q) True
incomplete likelihood is -491.37068071934556 on iteration 0
5-th start
Initial theta = [-1.9246683]
min=-0.3570181927074528
is_spd(weighted_Q) Fal

In [6]:
EM_theta1_deg = np.rad2deg(EM_theta1_rad)
print(f"Погрешность в смысле разности углов в градусах: {np.abs(EM_theta1_deg-theta1_deg)},")
print(f"Погрешность в смысле разности синусов углов: {np.abs(np.sin(EM_theta1_rad)-np.sin(theta1_rad))}.")

Погрешность в смысле разности углов в градусах: [0.39598665],
Погрешность в смысле разности синусов углов: [0.00605371].


In [7]:
EM_Q1-Q1

array([[ 0.18057048+0.00000000e+00j, -0.0293927 +3.33836543e-02j,
        -0.16644627+7.19667816e-02j, -0.05632357-1.83390416e-01j,
         0.09606395-1.22738444e-01j],
       [-0.0293927 -3.33836543e-02j, -0.05007532+0.00000000e+00j,
         0.07664818+1.28103967e-02j, -0.07484635+2.48991396e-02j,
        -0.12340658+3.60452195e-02j],
       [-0.16644627-7.19667816e-02j,  0.07664818-1.28103967e-02j,
         0.18371237+0.00000000e+00j,  0.01345501+1.81981776e-01j,
        -0.11784601+7.13562377e-02j],
       [-0.05632357+1.83390416e-01j, -0.07484635-2.48991396e-02j,
         0.01345501-1.81981776e-01j,  0.31831259+0.00000000e+00j,
         0.09694093+1.39819403e-01j],
       [ 0.09606395+1.22738444e-01j, -0.12340658-3.60452195e-02j,
        -0.11784601-7.13562377e-02j,  0.09694093-1.39819403e-01j,
         0.0704901 +1.74652275e-19j]])

In [8]:
print(np.round(Q1-EM_Q1, 3))

[[-0.181+0.j     0.029-0.033j  0.166-0.072j  0.056+0.183j -0.096+0.123j]
 [ 0.029+0.033j  0.05 +0.j    -0.077-0.013j  0.075-0.025j  0.123-0.036j]
 [ 0.166+0.072j -0.077+0.013j -0.184+0.j    -0.013-0.182j  0.118-0.071j]
 [ 0.056-0.183j  0.075+0.025j -0.013+0.182j -0.318+0.j    -0.097-0.14j ]
 [-0.096-0.123j  0.123+0.036j  0.118+0.071j -0.097+0.14j  -0.07 -0.j   ]]


In [9]:
for row in EM_Q1 - Q1:
    print(" ".join(f"{x:.3f}" for x in row))

0.181+0.000j -0.029+0.033j -0.166+0.072j -0.056-0.183j 0.096-0.123j
-0.029-0.033j -0.050+0.000j 0.077+0.013j -0.075+0.025j -0.123+0.036j
-0.166-0.072j 0.077-0.013j 0.184+0.000j 0.013+0.182j -0.118+0.071j
-0.056+0.183j -0.075-0.025j 0.013-0.182j 0.318+0.000j 0.097+0.140j
0.096+0.123j -0.123-0.036j -0.118-0.071j 0.097-0.140j 0.070+0.000j


In [10]:
for row in Q1:
    print(" ".join(f"{x:.3f}" for x in row))

0.225+0.000j -0.012-0.041j -0.089+0.034j -0.060-0.015j 0.060+0.009j
-0.012+0.041j 0.290+0.000j 0.036-0.057j -0.041-0.029j 0.042+0.069j
-0.089-0.034j 0.036+0.057j 0.117+0.000j 0.097-0.062j -0.087+0.007j
-0.060+0.015j -0.041+0.029j 0.097+0.062j 0.386+0.000j -0.083-0.152j
0.060-0.009j 0.042-0.069j -0.087-0.007j -0.083+0.152j 0.182-0.000j


In [11]:
for row in EM_Q1:
    print(" ".join(f"{x:.3f}" for x in row))

0.406+0.000j -0.041-0.008j -0.256+0.106j -0.116-0.198j 0.156-0.114j
-0.041+0.008j 0.240+0.000j 0.113-0.044j -0.115-0.004j -0.081+0.105j
-0.256-0.106j 0.113+0.044j 0.300+0.000j 0.111+0.120j -0.205+0.079j
-0.116+0.198j -0.115+0.004j 0.111-0.120j 0.705+0.000j 0.014-0.012j
0.156+0.114j -0.081-0.105j -0.205-0.079j 0.014+0.012j 0.252+0.000j


In [12]:
EM_theta1_deg

array([29.0438764])

In [13]:
theta1_deg

array([28.64788976])